In [9]:
import torch
import numpy as np
import pandas as pd
import skimage
import matplotlib.pyplot as plt
import imp
import core
import utils
%matplotlib inline
wd = 'E:/new_data/kaggle/planet/'
train_set = pd.read_csv(wd + 'train_v2.csv')
train_set['tags'] = train_set['tags'].apply(lambda x: x.split(' '))
test_set = pd.read_csv(wd+'sample_submission_v2.csv')
train_tags = ['clear', 'partly_cloudy', 'haze', 'cloudy', 'primary', 'agriculture', 'road', 'water',
             'cultivation', 'habitation', 'bare_ground', 'selective_logging', 'artisinal_mine', 
              'blooming', 'slash_burn', 'conventional_mine', 'blow_down']
label_map = {l: i for i, l in enumerate(train_tags)}
inv_label_map = {i: l for l, i in label_map.items()}
file_all = train_set['image_name'].values
y_tr = utils.get_y(train_set['tags'].values, label_map)
test_file_all = test_set['image_name'].values

nnts = np.load('../preds/170620_01_pred_ts.npy')
nntr = np.load('../preds/170620_01_pred_tr.npy')

gbts = np.load('../preds/gbm_all_pred_ts_170624_01.npy')
gbtr = np.load('../preds/gbm_all_pred_tr_170624_01.npy')

wnnts = np.load('./pred/modelweightedNN1_date710_pred_test.npy')
wnntr = np.load('./pred/modelweightedNN1_date710_pred_train.npy')

In [10]:
nntr.shape, nnts.shape, gbtr.shape, gbts.shape, wnnts.shape, wnntr.shape, y_tr.shape

((40479, 17),
 (61191, 17),
 (40479, 17),
 (61191, 17),
 (61191, 17),
 (40479, 17),
 (40479, 17))

In [44]:
def f2_score(y_true, y_prob, th):
    y_pred = np.zeros_like(y_prob, dtype = np.int8)
    y_pred = (y_prob > th)
    score = fbeta_score(y_true, y_pred, beta = 2)
    return score

# 找到最优th
def best_f2_score(y_true, y_pred, thresholds = np.arange(0, 1, 0.01)):
    x = 0
    best_score = 0
    for t in thresholds:
        score = f2_score(y_true, y_pred, t)
        if score > best_score:
            best_score = score
            x = t
    return x

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold

y_ts = np.zeros((gbts.shape[0], 17))
p_tr = np.zeros((gbtr.shape[0], 17))

for i_c in range(17):
    skf = StratifiedKFold(n_splits=5)
    x_tr = np.c_[nntr[:, i_c], gbtr[:, i_c], wnntr[:, i_c]]
    x_ts = np.c_[nnts[:, i_c], gbts[:, i_c], wnnts[:, i_c]]
    k_now = 0
    for i_tr, i_vl in skf.split(x_tr, y_tr[:, i_c]):
        lr = LogisticRegression(class_weight='balanced')
        lr.fit(x_tr[i_tr, :], y_tr[i_tr, i_c])
        p_tr[i_vl, i_c] = lr.predict_proba(x_tr[i_vl, :])[:,1]

    lr = LogisticRegression(class_weight='balanced')
    lr.fit(x_tr, y_tr[:, i_c])
    y_ts[:, i_c] = lr.predict_proba(x_ts)[:, 1]

In [12]:
th1 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(0, 1, 0.01))
th2 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(1, 0, -0.01))
th = (th1 + th2) / 2.0
utils.f2_score(y_tr, p_tr, th, num_classes=17)

0.92225287406242773

In [14]:
submit_df = utils.to_submit(y_ts, th, test_set, inv_label_map)
submit_df.to_csv('./submit/3model_stacking_date711_no1.csv', index=False)

In [73]:
y_ts[:, :4].argmax(axis=1)[:20]

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 1, 0, 3, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [76]:
submit_df = pu.to_submit_new(y_ts, th, mt.test_set)
submit_df.to_csv('./submit/170625_01.csv', index=False)

In [77]:
y_tr

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [99]:
yy = y_tr.copy()
def class2num(x):
    result = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        result[i] = (2 ** np.where(x[i, :] == 1)[0]).sum()
    return result
nums = class2num(yy)

In [116]:
import pandas as pd
%matplotlib inline
ddff = pd.DataFrame({'index': np.arange(nums.size), 'nums': nums})
ddff

,index,nums
0,0,20.0
1,1,177.0
2,2,17.0
3,3,17.0
4,4,625.0
5,5,148.0
6,6,433.0
7,7,20.0
8,8,305.0
9,9,369.0


In [120]:
count_ddff = ddff.groupby('nums').count()
count_ddff.loc[count_ddff['index']> 10, :]

,index
nums,
8.0,2089
17.0,13636
18.0,3630
20.0,1471
33.0,46
34.0,15
49.0,1626
50.0,692
52.0,172
